In [1]:
from taxcalc import *
from taxcalc.utils import *
from bokeh.io import show, output_notebook
from bokeh.layouts import column
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource, NumeralTickFormatter
from collections import OrderedDict
import copy
import pandas as pd
from notebookfunctions import distribution, index_list, percentile
output_notebook()

Loading BokehJS ...

In [8]:
# Data from IRS-SOI Tax Stats. Used for comparison
soi_stats = pd.read_csv('soi_stats.csv', index_col=0)  # Equivalent to tax-calc diagnostic table
soi_income = pd.read_csv('soi_income_stats.csv')  # Distribution of income items
soi_deductions = pd.read_csv('soi_deductions.csv', index_col=0)  # Itemized deductions
soi_deductions['index'] = soi_deductions.index

In [2]:
# Read in new PUF and associated data
gf = pd.read_csv('../taxdata/stage1/growfactors.csv')
wt = pd.read_csv('../taxdata/puf_stage2/puf_weights.csv')
puf = pd.read_csv('../taxdata/puf_data/puf.csv')
adj = pd.read_csv('../taxdata/puf_stage3/puf_ratios.csv', index_col=0)
adj = adj.transpose()

In [3]:
# base calculator
calc = Calculator(records=Records(), policy=Policy())
calc.advance_to_year(2014)
calc.calc_all()

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [4]:
# new calculator
growf = Growfactors('../taxdata/stage1/growfactors.csv')
rec = Records(puf, gfactors=growf,
              weights=wt, adjust_ratios=adj,
              start_year=2012)
pol = Policy(gfactors=growf, start_year=2012, num_years=16)
consump = Consumption(start_year=2012, num_years=16)
behave = Behavior(start_year=2012, num_years=16)
calc_new = Calculator(records=rec, policy=pol, consumption=consump, behavior=behave)
calc_new.advance_to_year(2014)
calc_new.calc_all()

You loaded data for 2012.
Tax-Calculator startup automatically extrapolated your data to 2012.


# New Disribution Table

In [5]:
create_distribution_table(calc_new, groupby='weighted_deciles',
                          result_type='weighted_sum', income_measure='c00100')

,num_returns_AMT,num_returns_ItemDed,s006,num_returns_StandardDed,refund,taxbc,c04600,c00100,iitax,aftertax_income,...,expanded_income,standard,payrolltax,c04470,c05800,combined,c04800,c62100,othertaxes,c09600
0,"16,572",0,"16,165,677","6,382,551","492,259,106","287,756","84,297,082,200","-60,400,221,264","3,635,428,031","117,120,728,875",...,"123,125,511,178","122,331,212,309","2,369,354,272",0,"4,097,272,843","6,004,782,303","6,898,156","-36,982,854,056","35,206,225","4,096,985,087"
1,"2,309","20,105","16,166,559","16,137,680","4,176,141,869","76,177,413","62,305,668,440","72,567,517,707","-3,836,210,770","149,502,822,547",...,"153,840,799,704","108,388,813,228","8,174,187,927","93,192,291","323,193,676","4,337,977,157","1,387,317,804","73,625,884,727","18,121,377","247,016,263"
2,"19,783","262,765","16,165,812","15,901,036","19,874,322,962","1,260,947,253","94,930,176,360","171,306,016,534","-18,302,906,264","252,739,794,782",...,"254,615,906,970","127,757,570,923","20,179,018,452","3,405,511,154","1,537,642,252","1,876,112,188","13,222,413,786","170,300,411,325","68,587,937","276,694,999"
3,"25,989","785,906","16,166,460","15,377,213","31,172,711,731","5,363,866,908","118,347,533,760","273,713,312,896","-25,926,771,049","348,431,888,220",...,"356,203,600,896","131,315,489,999","33,698,483,725","10,031,716,652","5,584,657,091","7,771,712,676","54,934,663,005","268,067,409,025","115,242,994","220,790,184"
4,"17,539","1,531,794","16,165,615","14,632,879","23,041,434,292","15,785,584,045","121,994,801,360","403,935,045,968","-9,061,891,850","444,694,424,075",...,"487,052,116,473","128,432,511,103","51,419,584,249","22,013,306,436","16,005,370,229","42,357,692,399","141,920,825,410","391,220,326,967","200,755,497","219,786,184"
5,"15,191","2,900,672","16,166,065","13,263,760","12,208,927,572","34,261,039,716","124,642,539,840","569,746,285,167","17,744,064,774","558,932,354,051",...,"650,349,846,212","118,919,131,689","73,673,427,387","43,547,036,856","34,508,703,743","91,417,492,161","285,155,792,715","545,803,896,150","370,005,427","247,664,027"
6,"23,992","4,812,021","16,166,231","11,354,149","2,648,144,640","61,998,145,140","129,319,056,040","793,275,742,204","53,011,483,076","722,925,580,561",...,"876,055,773,572","109,128,635,968","100,118,709,935","76,303,299,321","62,373,521,096","153,130,193,011","480,050,315,686","751,085,752,375","595,482,979","375,375,956"
7,"49,246","7,070,958","16,166,672","9,095,437","193,910,191","107,450,458,317","143,020,745,560","1,115,475,041,351","99,519,618,833","977,122,185,023",...,"1,211,000,191,088","98,633,186,650","134,358,387,233","129,677,264,229","108,095,114,690","233,878,006,065","746,795,300,424","1,044,875,566,626","775,338,110","644,656,373"
8,"153,760","10,050,924","16,165,379","6,112,504","33,955,199","181,601,438,201","164,416,252,400","1,630,299,641,347","173,617,069,716","1,385,141,262,667",...,"1,756,561,593,367","74,016,243,070","197,803,260,984","211,735,681,841","182,747,151,093","371,420,330,700","1,180,870,423,360","1,518,693,658,377","1,544,022,644","1,145,712,892"
9,"4,005,677","13,958,206","16,166,962","2,207,685","156,442","1,028,369,965,839","156,917,941,503","4,964,077,839,622","1,068,938,687,455","3,823,503,064,834",...,"5,233,784,724,502","27,553,936,211","341,342,972,214","592,973,739,705","1,061,090,753,122","1,410,281,659,669","4,188,513,306,358","4,705,921,314,113","30,649,540,332","32,720,787,283"


# Current Distribution Table

In [6]:
create_distribution_table(calc, groupby='weighted_deciles',
                          result_type='weighted_sum', income_measure='c00100')

,num_returns_AMT,num_returns_ItemDed,s006,num_returns_StandardDed,refund,taxbc,c04600,c00100,iitax,aftertax_income,...,expanded_income,standard,payrolltax,c04470,c05800,combined,c04800,c62100,othertaxes,c09600
0,"11,493",0,"16,220,752","3,708,850","906,029,892",0,"86,537,994,034","-149,726,787,864","2,361,457,503","31,540,778,847",...,"37,122,742,595","123,664,286,701","3,220,506,246",0,"3,184,071,522","5,581,963,748",0,"-136,818,460,696","86,953,480","3,184,071,522"
1,910,"16,978","16,223,564","16,193,482","3,091,156,478","29,889,716","62,800,517,303","49,153,922,700","-2,897,010,865","128,770,131,699",...,"131,490,747,997","102,703,346,069","5,617,627,163","54,523,814","168,832,395","2,720,616,298","697,441,193","49,487,976,905","25,782,677","138,942,679"
2,"27,438","127,058","16,222,306","16,085,102","15,739,892,911","617,680,780","88,758,571,182","143,134,086,177","-14,975,974,655","226,812,797,476",...,"227,737,509,498","125,770,900,218","15,900,686,677","1,491,389,937","694,218,447","924,712,022","6,585,562,650","142,336,044,721","95,003,962","76,537,667"
3,"56,262","778,872","16,222,064","15,443,192","30,601,982,626","4,084,319,153","114,942,622,376","249,213,176,722","-26,733,429,636","331,015,512,775",...,"333,707,338,575","130,019,629,204","29,425,255,435","10,086,591,886","4,321,455,307","2,691,825,799","42,100,726,133","242,729,143,343","117,955,756","237,136,154"
4,"32,681","1,515,580","16,223,014","14,706,390","25,346,402,224","13,612,657,115","121,756,634,656","382,321,791,610","-14,044,597,895","431,633,207,537",...,"464,944,700,633","127,392,012,174","47,356,090,991","23,212,520,833","13,810,348,792","33,311,493,096","124,657,921,396","367,965,705,687","249,777,902","197,691,676"
5,"22,179","2,798,053","16,222,499","13,424,385","14,281,999,026","32,428,593,755","124,038,481,638","553,710,452,123","12,629,098,555","552,297,913,647",...,"634,658,321,968","118,454,408,988","69,731,309,767","43,480,770,883","32,632,446,842","82,360,408,321","271,695,058,062","527,527,807,021","448,650,310","203,853,087"
6,"19,149","4,970,371","16,221,953","11,249,915","3,675,361,936","61,446,967,759","125,384,685,878","784,431,401,275","49,724,205,028","721,252,636,974",...,"867,110,514,253","104,753,187,850","96,133,672,251","81,764,884,123","61,802,074,423","145,857,877,279","475,188,293,251","738,301,676,883","718,152,651","355,106,663"
7,"64,292","7,234,590","16,223,206","8,984,526","1,386,607,192","108,830,829,123","139,235,241,390","1,112,500,051,832","97,332,040,887","975,809,232,814",...,"1,205,535,207,793","94,791,036,921","132,393,934,093","131,122,757,497","109,417,449,721","229,725,974,979","748,492,817,636","1,042,072,554,885","1,047,217,608","586,620,598"
8,"190,588","10,327,914","16,222,053","5,892,361","1,532,104,925","189,381,889,569","164,570,887,134","1,667,643,238,695","175,768,417,694","1,413,639,418,647",...,"1,793,082,486,885","69,933,991,672","203,674,650,545","222,515,689,982","190,120,233,842","379,443,068,239","1,211,402,555,603","1,550,734,759,269","1,601,565,184","738,344,273"
9,"4,404,713","14,207,267","16,223,209","2,015,145","873,195,538","1,123,569,557,383","154,909,947,388","5,228,312,503,191","1,161,499,625,317","3,959,322,111,835",...,"5,475,151,525,549","24,192,511,419","354,329,788,398","591,438,451,680","1,156,586,615,144","1,515,829,413,714","4,460,206,219,834","4,960,693,754,490","34,765,659,102","33,017,057,760"


# Diagnostic Table Comparison

In [7]:
new_diag = create_diagnostic_table(calc_new)
diag = create_diagnostic_table(calc)  # Current PUF

In [9]:
diag_data = pd.DataFrame()
diag_data['SOI'] = soi_stats['Value']
diag_data['New'] = new_diag[2014]
diag_data['Current'] = diag[2014]
diag_data['% Change'] = ((new_diag[2014] / diag[2014]) - 1) * 100
diag_data['New - SOI'] = map(abs, new_diag[2014] - diag_data['SOI'])
diag_data['Current - SOI'] = map(abs, diag[2014] - diag_data['SOI'])
diag_data

,SOI,New,Current,% Change,New - SOI,Current - SOI
Returns (#m),148.6,161.7,162.2,-0.3,13.1,13.6
AGI ($b),"9,771.0","9,934.0","10,020.7",-0.9,163.0,249.7
Itemizers (#m),44.0,41.4,42.0,-1.4,2.6,2.0
Itemized Deduction ($b),"1,206.7","1,089.8","1,105.2",-1.4,116.9,101.5
Standard Deduction Filers (#m),117.4,110.5,107.7,2.6,6.9,9.7
Standard Deduction ($b),876.2,969.4,925.0,4.8,93.2,48.8
Personal Exemption ($b),"1,121.6","1,144.8","1,113.8",2.8,23.2,7.8
Taxable Income ($b),"6,997.9","7,092.9","7,341.0",-3.4,95.0,343.1
Regular Tax ($b),nan,"1,436.2","1,534.0",-6.4,nan,nan
AMT Income ($b),nan,"9,432.6","9,485.0",-0.6,nan,nan


# Income Levels

In [10]:
inc_dict = OrderedDict()
inc_dict['New'] = [] 
inc_dict['Current'] = []
inc_dict['SOI'] = []
inc_dict['Pct Diff'] = []
inc_dict['New - SOI'] = []
inc_dict['Current - SOI'] = []
inc_list = ['WAS', 'Taxable Interest', 'Ordinary Dividends', 'Qualified Dividends', 'Business Income']
# Wage and salary
was_new = (calc_new.records.e00200 * calc_new.records.s006).sum()
inc_dict['New'].append(was_new)
was = (calc.records.e00200 * calc.records.s006).sum()
inc_dict['Current'].append(was)
soi_was = soi_income['WAS'].sum()
inc_dict['SOI'].append(soi_was)
new_was_diff = abs(was_new - soi_was)
inc_dict['New - SOI'].append(new_was_diff)
curr_was_diff = abs(was - soi_was)
inc_dict['Current - SOI'].append(curr_was_diff)
inc_dict['Pct Diff'].append((curr_was_diff / was) * 100)
# Interest income
int_new = (calc_new.records.e00300 * calc_new.records.s006).sum()
inc_dict['New'].append(int_new)
int_ = (calc.records.e00300 * calc.records.s006).sum()
inc_dict['Current'].append(int_)
int_soi = soi_income['INT'].sum()
inc_dict['SOI'].append(int_soi)
new_int_diff = abs(int_new - int_soi)
inc_dict['New - SOI'].append(new_int_diff)
curr_int_diff = abs(int_ - int_soi)
inc_dict['Current - SOI'].append(curr_int_diff)
inc_dict['Pct Diff'].append((curr_int_diff / int_) * 100)
# Ordinary dividends
odiv_new = (calc_new.records.e00600 * calc_new.records.s006).sum()
inc_dict['New'].append(odiv_new)
odiv = (calc.records.e00600 * calc.records.s006).sum()
inc_dict['Current'].append(odiv)
odiv_soi = soi_income['ODIV'].sum()
inc_dict['SOI'].append(odiv_soi)
new_odiv_diff = abs(odiv_new - odiv_soi)
inc_dict['New - SOI'].append(new_odiv_diff)
curr_odiv_diff = abs(odiv - odiv_soi)
inc_dict['Current - SOI'].append(curr_odiv_diff)
inc_dict['Pct Diff'].append((curr_odiv_diff / odiv) * 100)
# Qualified dividends
qdiv_new = (calc_new.records.e00650 * calc_new.records.s006).sum()
inc_dict['New'].append(qdiv_new)
qdiv = (calc.records.e00650 * calc.records.s006).sum()
inc_dict['Current'].append(qdiv)
qdiv_soi = soi_income['QDIV'].sum()
inc_dict['SOI'].append(qdiv_soi)
new_qdiv_diff = abs(qdiv_new - qdiv_soi)
inc_dict['New - SOI'].append(new_qdiv_diff)
curr_qdiv_diff = abs(qdiv - qdiv_soi)
inc_dict['Current - SOI'].append(curr_qdiv_diff)
inc_dict['Pct Diff'].append((curr_qdiv_diff / qdiv) * 100)

biz_new = (calc_new.records.e00900 * calc_new.records.s006).sum()
inc_dict['New'].append(biz_new)
biz = (calc.records.e00900 * calc.records.s006).sum()
inc_dict['Current'].append(biz)
biz_soi = soi_income['BIZ'].sum()
inc_dict['SOI'].append(biz_soi)
new_biz_diff = abs(biz_new - biz_soi)
inc_dict['New - SOI'].append(new_biz_diff)
curr_biz_diff = abs(biz - biz_soi)
inc_dict['Current - SOI'].append(curr_biz_diff)
inc_dict['Pct Diff'].append((curr_biz_diff / biz) * 100)

inc_df = pd.DataFrame.from_dict(inc_dict)
inc_df.index = inc_list
print "'Pct Diff' represents the percentage change between the current and new sums"
inc_df

'Pct Diff' represents the percentage change between the current and new sums


,New,Current,SOI,Pct Diff,New - SOI,Current - SOI
WAS,"6,784,960,449,585.9","6,820,912,371,695.9","6,785,880,966,000.0",0.5,"920,516,414.1","35,031,405,695.9"
Taxable Interest,"92,873,859,223.5","97,246,196,658.4","93,894,281,000.0",3.4,"1,020,421,776.5","3,351,915,658.4"
Ordinary Dividends,"254,702,179,872.0","270,137,086,776.2","254,702,232,000.0",5.7,"52,128.0","15,434,854,776.2"
Qualified Dividends,"196,858,639,935.9","198,124,994,913.5","192,447,708,000.0",2.9,"4,410,931,935.9","5,677,286,913.5"
Business Income,"317,256,662,774.3","309,444,223,964.1","317,248,783,000.0",2.5,"7,879,774.3","7,804,559,035.9"


# Distribution of Income

In [11]:
# Generate data for distribution plots
new_dist = pd.DataFrame()
cur_dist = pd.DataFrame()
new_was = distribution(calc_new.records.e00200, calc_new.records.s006, calc_new.records.c00100)
cur_was = distribution(calc.records.e00200, calc.records.s006, calc.records.c00100)
new_int = distribution(calc_new.records.e00300, calc_new.records.s006, calc_new.records.c00100)
cur_int = distribution(calc.records.e00300, calc.records.s006, calc.records.c00100)
new_odiv = distribution(calc_new.records.e00600, calc_new.records.s006, calc_new.records.c00100)
cur_odiv = distribution(calc.records.e00600, calc.records.s006, calc.records.c00100)
new_qdiv = distribution(calc_new.records.e00650, calc_new.records.s006, calc_new.records.c00100)
cur_qdiv = distribution(calc.records.e00650, calc.records.s006, calc.records.c00100)
new_biz = distribution(calc_new.records.e00900, calc_new.records.s006, calc_new.records.c00100)
cur_biz = distribution(calc.records.e00900, calc.records.s006, calc.records.c00100)
new_dist['WAS'] = new_was[1]
cur_dist['WAS'] = cur_was[1]
new_dist['INT'] = new_int[1]
cur_dist['INT'] = cur_int[1]
new_dist['ODIV'] = new_odiv[1]
cur_dist['ODIV'] = cur_odiv[1]
new_dist['QDIV'] = new_qdiv[1]
cur_dist['QDIV'] = cur_qdiv[1]
new_dist['BIZ'] = new_biz[1]
cur_dist['BIZ'] = cur_biz[1]
new_dist['AGI Bin'] = index_list()
cur_dist['AGI Bin'] = index_list()
new_dist['label'] = 'New'
cur_dist['label'] = 'Current'
# Create scatter plot objects
items_tups = [('WAS', 'WAS'), ('INT', 'Interest Income'), ('ODIV', 'Ordinary Dividends'),
              ('QDIV', 'Qualified Dividends'), ('BIZ', 'Business Income')]
soi_dist = pd.DataFrame()
for item in items_tups:
    soi_dist[item[0]] = (soi_income[item[0]] / soi_income[item[0]].sum())
soi_dist['AGI Bin'] = index_list()
soi_dist['label'] = 'SOI'
scatter_data = pd.concat([new_dist, cur_dist, soi_dist])
scatter_data_new = ColumnDataSource(scatter_data[scatter_data['label'] == 'New'])
scatter_data_cur = ColumnDataSource(scatter_data[scatter_data['label'] == 'Current'])
scatter_data_soi = ColumnDataSource(scatter_data[scatter_data['label'] == 'SOI'])

figure_list = []
for item in items_tups:
    title = 'Percent of Total {} by AGI Bin'.format(item[1])
    f = figure(title=title, x_range=index_list())
    f.xaxis.major_label_orientation = 45
    f.circle(x='AGI Bin', y=item[0], color='green', size=10, legend='New', alpha=0.5,
             source=scatter_data_new)
    f.circle(x='AGI Bin', y=item[0], color='red', size=10, legend='Current', alpha=0.5,
             source=scatter_data_cur)
    f.circle(x='AGI Bin', y=item[0], color='blue', size=10, legend='SOI', alpha=0.5,
             source=scatter_data_soi)
    f.legend.location = 'top_left'
    f.yaxis[0].formatter = NumeralTickFormatter(format="0.00%")
    hover = HoverTool(tooltips=[('Pct', '@{}%'.format(item[0]))])
    f.add_tools(hover)
    figure_list.append(f)
show(column(figure_list))

In [12]:
new_tot = pd.DataFrame()
cur_tot = pd.DataFrame()
new_tot['WAS'] = new_was[0]
cur_tot['WAS'] = cur_was[0]
new_tot['INT'] = new_int[0]
cur_tot['INT'] = cur_int[0]
new_tot['ODIV'] = new_odiv[0]
cur_tot['ODIV'] = cur_odiv[0]
new_tot['QDIV'] = new_qdiv[0]
cur_tot['QDIV'] = cur_qdiv[0]
new_tot['BIZ'] = new_biz[0]
cur_tot['BIZ'] = cur_biz[0]
new_tot['AGI Bin'] = index_list()
cur_tot['AGI Bin'] = index_list()
new_tot['label'] = 'New'
cur_tot['label'] = 'Current'
soi_income['AGI Bin'] = index_list()
soi_income['label'] = 'SOI'
total_data = pd.concat([new_tot, cur_tot, soi_income])
# Create scatter plot objects
items_tups = [('WAS', 'WAS'), ('INT', 'Interest Income'), ('ODIV', 'Ordinary Dividends'),
              ('QDIV', 'Qualified Dividends'), ('BIZ', 'Business Income')]

total_data_new = ColumnDataSource(total_data[total_data['label'] == 'New'])
total_data_cur = ColumnDataSource(total_data[total_data['label'] == 'Current'])
total_data_soi = ColumnDataSource(total_data[total_data['label'] == 'SOI'])

figure_list = []
for item in items_tups:
    title = 'Total {} by AGI Bin'.format(item[1])
    f = figure(title=title, x_range=index_list())
    f.xaxis.major_label_orientation = 45
    f.circle(x='AGI Bin', y=item[0], color='green', size=10, legend='New', alpha=0.5,
             source=total_data_new)
    f.circle(x='AGI Bin', y=item[0], color='red', size=10, legend='Current', alpha=0.5,
             source=total_data_cur)
    f.circle(x='AGI Bin', y=item[0], color='blue', size=10, legend='SOI', alpha=0.5,
             source=total_data_soi)
    f.legend.location = 'top_left'
    f.yaxis[0].formatter = NumeralTickFormatter(format='$0.00a')
    hover = HoverTool(tooltips=[('Total', '@{}{}'.format(item[0], '{$0.00a}'))])
    f.add_tools(hover)
    figure_list.append(f)
show(column(figure_list))

In [13]:
deductions_new = {'Medical Expenses': (calc_new.records.e17500[calc_new.records.c04470 > 0] *
                                        calc_new.records.s006[calc_new.records.c04470 > 0]).sum(),
                   'State and Local Taxes':  (calc_new.records.e18400[calc_new.records.c04470 > 0] *
                                              calc_new.records.s006[calc_new.records.c04470 > 0]).sum(),
                   'Real Estate Taxes':  (calc_new.records.e18500[calc_new.records.c04470 > 0] *
                                          calc_new.records.s006[calc_new.records.c04470 > 0]).sum(),
                   'Interest Paid':  (calc_new.records.e19200[calc_new.records.c04470 > 0] *
                                       calc_new.records.s006[calc_new.records.c04470 > 0]).sum(),
                   'Charitable Cash Contributions': (calc_new.records.e19800[calc_new.records.c04470 > 0] *
                                                     calc_new.records.s006[calc_new.records.c04470 > 0]).sum(),
                   'Charitable Non-Cash Contributions': (calc_new.records.e20100[calc_new.records.c04470 > 0] *
                                                         calc_new.records.s006[calc_new.records.c04470 > 0]).sum(),
                   'Total Misc. Expenses':  (calc_new.records.e20400[calc_new.records.c04470 > 0] *
                                             calc_new.records.s006[calc_new.records.c04470 > 0]).sum(),
                   'Net Casualty or Loss': (calc_new.records.g20500[calc_new.records.c04470 > 0] *
                                            calc_new.records.s006[calc_new.records.c04470 > 0]).sum()}
ded_new_df = pd.DataFrame.from_dict(deductions_new, 'index')
ded_new_df.columns = ['Total']
ded_new_df['source'] = 'New'

deductions = {'Medical Expenses': (calc.records.e17500[calc.records.c04470 > 0] *
                                   calc.records.s006[calc.records.c04470 > 0]).sum(),
              'State and Local Taxes':  (calc.records.e18400[calc.records.c04470 > 0] *
                                         calc.records.s006[calc.records.c04470 > 0]).sum(),
              'Real Estate Taxes':  (calc.records.e18500[calc.records.c04470 > 0] *
                                     calc.records.s006[calc.records.c04470 > 0]).sum(),
              'Interest Paid':  (calc.records.e19200[calc.records.c04470 > 0] *
                                 calc.records.s006[calc.records.c04470 > 0]).sum(),
              'Charitable Cash Contributions': (calc.records.e19800[calc.records.c04470 > 0] *
                                                calc.records.s006[calc.records.c04470 > 0]).sum(),
              'Charitable Non-Cash Contributions': (calc.records.e20100[calc.records.c04470 > 0] *
                                                    calc.records.s006[calc.records.c04470 > 0]).sum(),
              'Total Misc. Expenses':  (calc.records.e20400[calc.records.c04470 > 0] *
                                        calc.records.s006[calc.records.c04470 > 0]).sum(),
              'Net Casualty or Loss': (calc.records.g20500[calc.records.c04470 > 0] *
                                       calc.records.s006[calc.records.c04470 > 0]).sum()}
ded_df = pd.DataFrame.from_dict(deductions, 'index')
ded_df.columns = ['Total']
ded_df['source'] = 'Current'
soi_deductions['source'] = 'SOI'

ded_full_df = pd.concat([ded_new_df, ded_df, soi_deductions])

In [14]:
ded_cds_new = ColumnDataSource(ded_full_df[ded_full_df['source'] == 'New'])
ded_cds_new.add(data=[i - .5 for i in range(1, 9)], name='xaxis')
ded_cds_cur = ColumnDataSource(ded_full_df[ded_full_df['source'] == 'Current'])
ded_cds_cur.add(data=[i - .25 for i in range(1, 9)], name='xaxis')
ded_cds_soi = ColumnDataSource(ded_full_df[ded_full_df['source'] == 'SOI'])
ded_cds_soi.add(data=[i - .75 for i in range(1, 9)], name='xaxis')

'xaxis'

In [15]:
xaxis_list = ['Interest Paid', 'Total Misc. Expenses', 'Medical Expenses',
              'Charitable Non-Cash Contributions', 'Real Estate Taxes',
              'State and Local Taxes', 'Charitable Cash Contributions',
              'Net Casualty or Loss']
f = figure(title='Itemized Deduction Totals', x_range=xaxis_list, width=800)
#f = figure(title='Itemized Deduction Totals')
f.xaxis.major_label_orientation = 45
f.vbar(x='xaxis', bottom=0, top='Total', legend='New',
       color='green', alpha=0.5, line_alpha=1,
       width=0.25, source=ded_cds_new)
f.vbar(x='xaxis', bottom=0, top='Total', legend='Current', color='red',
       alpha=0.5, line_alpha=1,
       width=0.25, source=ded_cds_cur)
f.vbar(x='xaxis', bottom=0, top='Total', legend='SOI', color='blue',
       alpha=0.5, line_alpha=1,
       width=0.25, source=ded_cds_soi)
hover = HoverTool(tooltips=[('Total', '@Total{$0.00a}')])
f.yaxis[0].formatter = NumeralTickFormatter(format='$0.00a')
f.add_tools(hover)

show(f)

In [16]:
ded_error_df = pd.DataFrame()
ded_error_df['Difference: New'] = ded_new_df['Total'] - soi_deductions['Total']
ded_error_df['% Difference: New'] = 100 * ded_error_df['Difference: New'] / soi_deductions['Total']
ded_error_df['Difference: Current'] = ded_df['Total'] - soi_deductions['Total']
ded_error_df['% Difference: Current'] = 100 * ded_error_df['Difference: Current'] / soi_deductions['Total']
print 'Error in Itemized Deductions Relative to SOI Totals'
ded_error_df

Error in Itemized Deductions Relative to SOI Totals


,Difference: New,% Difference: New,Difference: Current,% Difference: Current
Charitable Cash Contributions,"-4,909,845,726.0",-3.2,"-4,697,400,311.8",-3.0
Charitable Non-Cash Contributions,"-20,258,359,383.2",-31.0,"-31,554,076,082.7",-48.3
Interest Paid,"-12,479,893,896.8",-4.1,"-7,473,328,319.6",-2.4
Medical Expenses,"-17,892,823,670.3",-13.9,"-9,751,487,495.3",-7.6
Net Casualty or Loss,"5,005,943,747.1",227.1,"2,251,393,716.6",102.1
Real Estate Taxes,"-6,525,885,181.3",-3.6,"17,200,103,389.4",9.5
State and Local Taxes,"-9,342,675,154.4",-2.8,"-16,242,417,254.8",-4.9
Total Misc. Expenses,"7,169,575,023.8",5.7,"3,967,640,537.5",3.1


In [42]:
# Create DataFrame subset of calculator records for the new weights
item_df_new = pd.DataFrame({'e00200': calc_new.records.e00200,
                             's006': calc_new.records.s006,
                             'c04470': calc_new.records.c04470})
item_df_new['itemizer'] = np.where(calc_new.records.c04470 > 0, 1, 0)
item_participation_rt_new = percentile(item_df_new, 'itemizer', 100, 'e00200', 's006')
# Create DataFrame subset for current weights
item_df = pd.DataFrame({'e00200': calc.records.e00200,
                        's006': calc.records.s006,
                        'c04470': calc.records.c04470})
item_df['itemizer'] = np.where(calc.records.c04470 > 0, 1, 0)
item_participation_rt = percentile(item_df, 'itemizer', 100, 'e00200', 's006')
item_index = item_participation_rt_new.index

# Create column data sources for plotting
item_cds_new = ColumnDataSource({'rate': item_participation_rt_new,
                                 'index': item_participation_rt_new.index})
item_cds_cur = ColumnDataSource({'rate': item_participation_rt,
                                 'index': item_participation_rt_new.index})

In [44]:
hover_ded = HoverTool(tooltips=[('Percentile', '$index'), ('Itemizing Rate', '@rate{0.00%}')])
f = figure(title='Itemizing Rate by Wage Percentile',
           x_axis_label='Wage Percentile', tools=[hover_ded, 'save', 'reset', 'zoom_in', 'zoom_out'])
f.line('index', 'rate', legend='New', line_width=2, source=item_cds_new)
f.line('index', 'rate', legend='Current', color='red', line_width=2, source=item_cds_cur)
f.legend.location = 'top_left'
f.yaxis[0].formatter = NumeralTickFormatter(format='0.00%')
show(f)

In [19]:
# Average Itemized Deduction by percentile
itemizers_new = copy.deepcopy(item_df_new[item_df_new['c04470'] > 0])
item_mean_new = percentile(itemizers_new, 'c04470', 100, 'e00200', 's006')
itemizers = copy.deepcopy(item_df[item_df['c04470'] > 0])
item_mean = percentile(itemizers, 'c04470', 100, 'e00200', 's006')
# Create column data source for plotting
item_cds_new = ColumnDataSource({'mean': item_mean_new,
                                 'xaxis': item_index})
item_cds_cur = ColumnDataSource({'mean': item_mean,
                                 'xaxis': item_index})

In [40]:
hover = HoverTool(tooltips=[('Percentile', '@xaxis'), ('Mean', '@mean{$0.00a}')])
f = figure(title='Mean Itemized Deduction Total Among Itemizers',
           x_axis_label='Wage Percentile', tools=[hover, 'save', 'reset', 'zoom_in', 'zoom_out'])
f.line('xaxis', 'mean', legend='New', source=item_cds_new, line_width=2)
f.line('xaxis', 'mean', legend='Current', color='red', source=item_cds_cur, line_width=2)
f.legend.location = 'top_left'
f.yaxis[0].formatter = NumeralTickFormatter(format='$0.00a')
show(f)

# Tax Liability by Year

In [21]:
liabilities_current = {'Income Tax': [],
                       'Payroll Tax': [],
                       'Combined Tax': [],
                       'source': []}
liabilities_new = {'Income Tax': [],
                   'Payroll Tax': [],
                   'Combined Tax': [],
                   'source': []}
years = []
for year in range(2015, 2028):
    years.append(year)
    calc.advance_to_year(year)
    calc_new.advance_to_year(year)
    calc.calc_all()
    calc_new.calc_all()
    liabilities_current['Income Tax'].append((calc.records.s006 * calc.records.iitax).sum())
    liabilities_current['Payroll Tax'].append((calc.records.s006 * calc.records.payrolltax).sum())
    liabilities_current['Combined Tax'].append((calc.records.s006 * calc.records.combined).sum())
    liabilities_new['Income Tax'].append((calc_new.records.s006 * calc_new.records.iitax).sum())
    liabilities_new['Payroll Tax'].append((calc_new.records.s006 * calc_new.records.payrolltax).sum())
    liabilities_new['Combined Tax'].append((calc_new.records.s006 * calc_new.records.combined).sum())
    liabilities_current['source'].append('Current')
    liabilities_new['source'].append('New')
liabilities_new['years'] = years
liabilities_current['years'] = years

In [22]:
# column data sources for plotting
liabilities_cds_new = ColumnDataSource(liabilities_new)
liabilities_cds_cur = ColumnDataSource(liabilities_current)

In [33]:
f = figure(title='Tax Liabilities 2015-2027')
f.width = 800
f.height = 500
f.line('years', 'Income Tax', color='blue', line_width=2, legend='Income-Current',
       source=liabilities_cds_cur)
f.line('years', 'Income Tax', color='cyan', line_width=2, legend='Income-New',
       source=liabilities_cds_new)
f.line('years', 'Payroll Tax', color='purple', line_width=2, legend='Payroll-Current',
       source=liabilities_cds_cur)
f.line('years', 'Payroll Tax', color='magenta', line_width=2, legend='Payroll-New',
       source=liabilities_cds_new)
f.line('years', 'Combined Tax', color='green', line_width=2, legend='Combined-Current',
       source=liabilities_cds_cur)
f.line('years', 'Combined Tax', color='lime', line_width=2, legend='Combined-New',
       source=liabilities_cds_new)
f.legend.location='top_left'
f.xaxis[0].ticker.desired_num_ticks = 12
f.xaxis.minor_tick_line_color = None
f.yaxis[0].formatter = NumeralTickFormatter(format='$0.00a')
c1 = f.circle('years', 'Income Tax', color='blue', size=5, legend='Income-Current',
              source=liabilities_cds_cur)
c2 = f.circle('years', 'Income Tax', color='cyan', size=5, legend='Income-New',
              source=liabilities_cds_new)
c3 = f.circle('years', 'Payroll Tax', color='purple', size=5, legend='Payroll-Current',
              source=liabilities_cds_cur)
c4 = f.circle('years', 'Payroll Tax', color='magenta', size=5, legend='Payroll-New',
              source=liabilities_cds_new)
c5 = f.circle('years', 'Combined Tax', color='green', size=5, legend='Combined-Current',
              source=liabilities_cds_cur)
c6 = f.circle('years', 'Combined Tax', color='lime', size=5, legend='Combined-New',
              source=liabilities_cds_new)
hover = HoverTool(tooltips=[('Calculator', '@source'), ('Year', '@years'),
                            ('Income Tax', '$y{$0.00a}')],
                  renderers=[c1, c2])
hover2 = HoverTool(tooltips=[('Calculator', '@source'), ('Year', '@years'),
                             ('Payroll Tax', '$y{$0.00a}')],
                  renderers=[c3, c4])
hover3 =HoverTool(tooltips=[('Calculator', '@source'), ('Year', '@years'),
                            ('Combined Tax', '$y{$0.00a}')],
                  renderers=[c5, c6])
f.add_tools(hover, hover2, hover3)

show(f)

In [24]:
ldf_current = pd.DataFrame(liabilities_current, index=years)
ldf_current.drop(['source', 'years'], axis=1, inplace=True)
print 'Current Tax Liabilities by Year'
ldf_current

Current Tax Liabilities by Year


,Combined Tax,Income Tax,Payroll Tax
2015,"2,580,952,229,934.0","1,580,105,276,837.2","1,000,846,953,096.9"
2016,"2,652,676,860,126.7","1,615,051,720,045.1","1,037,625,140,081.5"
2017,"2,813,546,326,543.2","1,720,999,668,174.9","1,092,546,658,368.3"
2018,"2,937,654,024,096.4","1,795,828,233,432.2","1,141,825,790,664.1"
2019,"3,045,538,636,999.4","1,858,467,869,104.0","1,187,070,767,895.5"
2020,"3,154,058,759,275.7","1,924,670,545,585.3","1,229,388,213,690.6"
2021,"3,273,926,547,197.1","1,999,801,411,738.6","1,274,125,135,458.4"
2022,"3,409,445,728,811.7","2,086,327,698,559.6","1,323,118,030,252.1"
2023,"3,556,676,832,883.3","2,181,320,492,211.6","1,375,356,340,671.6"
2024,"3,711,096,056,404.3","2,281,099,251,848.1","1,429,996,804,556.1"


In [25]:
ldf_new = pd.DataFrame(liabilities_new, index=years)
ldf_new.drop(['source', 'years'], axis=1, inplace=True)
print 'New Tax Liabilities by Year'
ldf_new

New Tax Liabilities by Year


,Combined Tax,Income Tax,Payroll Tax
2015,"2,498,372,690,086.6","1,493,015,947,278.0","1,005,356,742,808.5"
2016,"2,589,256,374,128.5","1,535,733,054,777.4","1,053,523,319,351.2"
2017,"2,730,738,944,355.0","1,628,628,096,082.2","1,102,110,848,272.7"
2018,"2,853,447,823,914.0","1,702,036,958,664.4","1,151,410,865,249.6"
2019,"2,960,726,425,817.8","1,763,895,223,993.6","1,196,831,201,824.3"
2020,"3,068,168,620,013.5","1,828,973,913,136.6","1,239,194,706,876.9"
2021,"3,186,237,976,179.1","1,901,970,360,621.8","1,284,267,615,557.3"
2022,"3,318,808,407,418.8","1,984,984,088,551.5","1,333,824,318,867.2"
2023,"3,463,241,439,470.8","2,076,790,707,540.5","1,386,450,731,930.3"
2024,"3,613,590,295,401.5","2,172,222,457,276.8","1,441,367,838,124.7"


In [26]:
print 'Difference in Tax Liabilities by Year'
diff_df = ldf_new - ldf_current
diff_df = diff_df.append(diff_df.sum(), ignore_index=True)
years.append('Total')
diff_df.index = years
diff_df

Difference in Tax Liabilities by Year


,Combined Tax,Income Tax,Payroll Tax
2015,"-82,579,539,847.4","-87,089,329,559.2","4,509,789,711.7"
2016,"-63,420,485,998.1","-79,318,665,267.7","15,898,179,269.6"
2017,"-82,807,382,188.3","-92,371,572,092.7","9,564,189,904.4"
2018,"-84,206,200,182.3","-93,791,274,767.8","9,585,074,585.5"
2019,"-84,812,211,181.5","-94,572,645,110.4","9,760,433,928.8"
2020,"-85,890,139,262.3","-95,696,632,448.7","9,806,493,186.3"
2021,"-87,688,571,017.9","-97,831,051,116.8","10,142,480,098.9"
2022,"-90,637,321,393.0","-101,343,610,008.1","10,706,288,615.1"
2023,"-93,435,393,412.5","-104,529,784,671.0","11,094,391,258.6"
2024,"-97,505,761,002.8","-108,876,794,571.3","11,371,033,568.5"


In [27]:
print 'Pct. Change in Tax Liabilities by Year'
((ldf_new - ldf_current) / ldf_current) * 100

Pct. Change in Tax Liabilities by Year


,Combined Tax,Income Tax,Payroll Tax
2015,-3.2,-5.5,0.5
2016,-2.4,-4.9,1.5
2017,-2.9,-5.4,0.9
2018,-2.9,-5.2,0.8
2019,-2.8,-5.1,0.8
2020,-2.7,-5.0,0.8
2021,-2.7,-4.9,0.8
2022,-2.7,-4.9,0.8
2023,-2.6,-4.8,0.8
2024,-2.6,-4.8,0.8


# Income Distribution

In [29]:
calc_dist = Calculator(records=Records(), policy=Policy())
calc_dist.advance_to_year(2014)
calc_dist.calc_all()

# new calculator
growfd = Growfactors('../taxdata/stage1/growfactors.csv')
red = Records(puf, gfactors=growfd,
              weights=wt, adjust_ratios=adj,
              start_year=2012)
pod = Policy(gfactors=growfd, start_year=2012, num_years=16)
consumpd = Consumption(start_year=2012, num_years=16)
behaved = Behavior(start_year=2012, num_years=16)
calc_dist_new = Calculator(records=red, policy=pod, consumption=consumpd, behavior=behaved)
calc_dist_new.advance_to_year(2014)
calc_dist_new.calc_all()

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
You loaded data for 2012.
Tax-Calculator startup automatically extrapolated your data to 2012.


In [45]:
income_df = pd.DataFrame({'c00100': calc_dist.records.c00100,
                          's006': calc_dist.records.s006})
income_df_new = pd.DataFrame({'c00100': calc_dist_new.records.c00100,
                              's006': calc_dist_new.records.s006})
mean_inc = percentile(income_df, 'c00100', 100, 'c00100', 's006')
mean_inc_new = percentile(income_df_new, 'c00100', 100, 'c00100', 's006')
income_cds = ColumnDataSource({'mean': mean_inc,
                               'percentile': item_index,
                               'source': ['Current'] * len(mean_inc)})
income_cds_new = ColumnDataSource({'mean': mean_inc_new,
                                   'percentile': item_index,
                                   'source': ['New'] * len(mean_inc_new)})

In [46]:
f = figure(title='Average AGI by AGI Percentile - 2014')
f.line(x='percentile', y='mean', source=income_cds, legend='Current', line_width=2)
f.line(x='percentile', y='mean', color='red', source=income_cds_new, legend='New', line_width=2)
f.legend.location='top_left'
f.yaxis[0].formatter = NumeralTickFormatter(format='$0.00a')
hover = HoverTool(tooltips=[('Percentile', '@percentile'), ('Avg. AGI', '@mean{$0.00a}'),
                            ('Calculator', '@source')])
f.add_tools(hover)
show(f)